In [2]:
import scripts.stock_plots as stock_plots
# import scripts.stock_analysis as stock_ana
import plotly.graph_objects as go

from datetime import datetime
import pandas as pd
import numpy as np
import yfinance as yf

pd.set_option('mode.chained_assignment', None)  # Ignore the warning

In [3]:
def download_data(stocks):
    stocks_str = " ".join(stocks)
    return yf.Tickers(stocks_str)

In [4]:
big = ["SMCI", "AAPL", "GOOG", "AMZN", "TSLA"]
stock_data = download_data(big)


In [5]:
def multi_forecast(df, days):
    df["priceRange"] = df.impliedVolatility * df.strike * np.sqrt(days/365)
    df["upper"] = (df.strike + df.priceRange).astype(int)
    df["lower"] = (df.strike - df.priceRange).astype(int)
    return df

def multi_op_chain(op_chain, days, x_range):
    op_chain = op_chain[op_chain.openInterest > op_chain.openInterest.mean()]
    op_chain = multi_forecast(op_chain, days)
    
    # range_analysis = np.zeros(op_chain.upper.max() + 1)
    range_analysis = np.zeros(x_range[1])

    for _, row in op_chain.iterrows():
        range_analysis[row.lower:row.upper] += 1


    # Create a bar chart
    fig = go.Figure(data=go.Bar(y=range_analysis))

    # Update layout
    fig.update_layout(
        title='Bar Chart',
        xaxis=dict(title='Category'),
        yaxis=dict(title='Value')
    )

    fig.update_xaxes(range=x_range)  # Assuming x values are indices starting from 0


    # Show plot
    fig.show()

    return range_analysis


In [6]:
selected = "GOOG"
op_dates = stock_data.tickers[selected].options

last_close = stock_data.tickers[selected].history(period=f"1d")["Close"].values[0]
x_range = [int(last_close * 0.5), int(last_close * 1.5)]

for i in [0,1,2,3]:
    days = (datetime.strptime(op_dates[i], '%Y-%m-%d') - datetime.now()).days

    callop = multi_op_chain(stock_data.tickers[selected].option_chain(op_dates[i]).calls, days, x_range)
    putop = multi_op_chain(stock_data.tickers[selected].option_chain(op_dates[i]).puts, days, x_range)

    comb = putop + callop
    fig = go.Figure(data=go.Bar(y=comb))

    # Update layout
    fig.update_layout(
        title='Combined',
        xaxis=dict(title='Category'),
        yaxis=dict(title='Value')
    )

    fig.update_xaxes(range=x_range)


    # Show plot
    fig.show()


In [7]:
def std_cal(strike, iv, days):
    std = iv * strike * np.sqrt(days/365)
    return strike+std, strike-std

def multi_forecast(df, days):
    df["priceRange"] = df.impliedVolatility * df.strike * np.sqrt(days/365)
    df["upper"] = (df.strike + df.priceRange).astype(int)
    df["lower"] = (df.strike - df.priceRange).astype(int)
    return df

def range_collection(op_chain):
    range_analysis = np.zeros(2000)

    for _, row in op_chain.iterrows():
        range_analysis[row.lower:row.upper] += 1
    
    range_idx = np.where(range_analysis == (np.max(range_analysis) - 1))
    return range_idx

big = ["SMCI", "AAPL", "GOOG", "AMZN", "TSLA", "AI"]
stock_data = download_data(big)
op_dates = stock_data.tickers[selected].options

for selected in big:
    stock_price = stock_data.tickers[selected].history(period=f"400d")

    stock_plot = stock_plots.PlotInfo(stock_price)
    candle = stock_plot.generate_candle_plot(p2p_order=4)
    candle.update_layout(title={"text": selected})

    forecast = {"iv_upper": [],
                        "iv_lower": [],
                        "call_upper": [],
                        "call_lower": [],
                        "put_upper": [],
                        "put_lower": []}

    # up = []
    # low = []
    date = []
    for i in [0,1,2,3,4,5]:
        tick = selected
        days = (datetime.strptime(op_dates[i], '%Y-%m-%d') - datetime.now()).days

        call_chain = stock_data.tickers[tick].option_chain(op_dates[i]).calls
        idx = sum(call_chain["inTheMoney"]) - 1
        strike = call_chain.iloc[idx]["strike"]
        iv = call_chain.iloc[idx]["impliedVolatility"]

        call_chain = call_chain[call_chain.openInterest > call_chain.openInterest.mean()]
        call_chain = multi_forecast(call_chain, days)
        range_idx = range_collection(call_chain)
        
        
        
        put_chain = stock_data.tickers[tick].option_chain(op_dates[i]).puts
        idx1 = -sum(put_chain["inTheMoney"])
        strike1 = put_chain.iloc[idx1]["strike"]
        iv1 = put_chain.iloc[idx1]["impliedVolatility"]

        put_chain = put_chain[put_chain.openInterest > put_chain.openInterest.mean()]
        put_chain = multi_forecast(put_chain, days)
        range_idx1 = range_collection(put_chain)
        

        strike = (strike + strike1) / 2
        iv = (iv + iv1) / 2
        ans = std_cal(strike, iv, days)

        
        if i == 0:
            # date.append(date.append(op_dates[i]))
            # up.append(strike)
            # low.append(strike)
            date.append(stock_price.index[-1])
            forecast = {"iv_upper": [strike],
                        "iv_lower": [strike],
                        "call_upper": [strike],
                        "call_lower": [strike],
                        "put_upper": [strike],
                        "put_lower": [strike]}
            

        date.append(op_dates[i])
        forecast["iv_upper"].append(ans[0])
        forecast["iv_lower"].append(ans[1])
        forecast["call_upper"].append(np.max(range_idx))
        forecast["call_lower"].append(np.min(range_idx))
        forecast["put_upper"].append(np.max(range_idx1))
        forecast["put_lower"].append(np.min(range_idx1))
        # up.append(ans[0])
        # low.append(ans[1])
    
    candle.add_trace(go.Scatter(x=date, y=forecast["iv_upper"], mode='lines+markers', line_color="grey", name='pred'), row=1, col=1)
    candle.add_trace(go.Scatter(x=date, y=forecast["iv_lower"], mode='lines+markers', line_color="grey", showlegend=False), row=1, col=1)
    candle.add_trace(go.Scatter(x=date, y=forecast["call_upper"], mode='lines+markers', line_color="green", name='call'), row=1, col=1)
    candle.add_trace(go.Scatter(x=date, y=forecast["call_lower"], mode='lines+markers', line_color="green", showlegend=False), row=1, col=1)
    candle.add_trace(go.Scatter(x=date, y=forecast["put_upper"], mode='lines+markers', line_color="red", name='put'), row=1, col=1)
    candle.add_trace(go.Scatter(x=date, y=forecast["put_lower"], mode='lines+markers', line_color="red", showlegend=False), row=1, col=1)

    candle.update_layout(xaxis=dict(rangebreaks=[dict(bounds=["sat", "mon"])]))

    candle.show()
    # print(forecast)